In [2]:
import os
import requests
import pandas as pd
import io
import csv
import glob


In [8]:
path = '/opt/data/GIS-Census/florida'
filename = path + '/census-post-matching-unmatched.csv'
dfUnmatched = pd.read_csv(filename, sep=',', engine='python')
print(f"Unmatched count: {len(dfUnmatched)}")
dfUnmatched['State'] = 'FL'
dfUnmatched

Unmatched count: 98249


,VoterID,Voter_Name,Last_Name,First_Name,Middle_Name,Name_Style,Formatted_Address,City_State,Zip_Country,plus4,...,MATCH_TYPE,ADDRESS_OUT,LONG_LAT,TIGER_EDGE,STREET_SIDE,FIPS_STATE,FIPS_COUNTY,CENSUS_TRACT,CENSUS_BLOCK,State
0,117058484,"Alsobrook, Andrew C",Alsobrook,Andrew,Clayton,NaN,2315 W CLOVELLY LN,SAINT AUGUSTINE FL,32092,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
1,116997977,"Anderson, Whitney N",Anderson,Whitney,Nicole,NaN,4369 S Quebec St Apt 6110,Denver CO,80237,2655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
2,126482969,"Arango, Juan P",Arango,Juan,Pablo,NaN,PO BOX 1344,EAST PALATKA FL,32131,1344,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
3,124465744,"Aviles, David A",Aviles,David,A,NaN,35855 Landon Ln,Murrieta CA,92562,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
4,116117059,"Bankston, Dominique BS",Bankston,Dominique,Breanna Samon,NaN,411 WILDCAT WAY APT 307,KNOXVILLE TN,37918,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98244,120192803,"Wolos, Charlene C",Wolos,Charlene,Christi,NaN,6034 Triumph Ln W,Jacksonville FL,32244,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
98245,103411820,"Wolos III, Frank J",Wolos,Frank,Joseph,III,6034 Triumph Ln W,Jacksonville FL,32244,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
98246,103411821,"Wolos, Janet H",Wolos,Janet,H,NaN,6034 Triumph Ln W,Jacksonville FL,32244,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL
98247,122097518,"Wolos, Jeremy L",Wolos,Jeremy,Luke,NaN,6034 Triumph Ln W,Jacksonville FL,32244,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL


In [9]:
df_addresses = dfUnmatched.filter(items=['VoterID', 'Residence_Address', 'City_Name', 'State', 'Zip_Code'])
df_addresses

,VoterID,Residence_Address,City_Name,State,Zip_Code
0,117058484,7936 Woodleigh Dr S,Jacksonville,FL,32211
1,116997977,7326 Arble Dr,Jacksonville,FL,32211
2,126482969,1350 Eddy Rd,Jacksonville,FL,32211
3,124465744,1112 Carlotta Rd W,Jacksonville,FL,32211
4,116117059,1505 Carlotta Rd W,Jacksonville,FL,32211
...,...,...,...,...,...
98244,120192803,6034 Triumph Ln W,Jacksonville,FL,32244
98245,103411820,6034 Triumph Ln W,Jacksonville,FL,32244
98246,103411821,6034 Triumph Ln W,Jacksonville,FL,32244
98247,122097518,6034 Triumph Ln W,Jacksonville,FL,32244


In [13]:
# Define the chunk size
chunk_size = 5000

# Get the number of chunks
num_chunks = len(dfUnmatched) // chunk_size + (1 if len(dfUnmatched) % chunk_size > 0 else 0)

# Save each chunk to a separate CSV file
for i in range(num_chunks):
    start_row = i * chunk_size
    end_row = start_row + chunk_size
    chunk = df_addresses.iloc[start_row:end_row]
    
    # Save the chunk to a CSV file
    chunk.to_csv(f'data_chunk_{i+1}.csv', index=False, header=False)

print(f'{num_chunks} CSV files have been created.')

20 CSV files have been created.


In [17]:
# outfile = f"{path}/census-post-matching-unmatched.csv"
# df_unmatched.to_csv(outfile, index=False)

In [11]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}

In [15]:
for i in range(3,21):       # adjust these params, hacked start/end during trials
    data_chunk = f"{path}/data_chunk_{i+1}.csv"
    print(f"Data chunk: {data_chunk}")
    files = {'addressFile': (filename, open(data_chunk, 'rb'), 'text/csv')}
    s = requests.post(url, files=files, data=payload, timeout=None)
    
    print("\t\tGot reponse from Census API")
    response_chunk_name = f"{path}/data_result_{i+1}.csv"
    response_chunk_file = open(response_chunk_name, 'wb')
    response_chunk_file.write(s.content)
    response_chunk_file.close()


Data chunk: /opt/data/GIS-Census/florida/data_chunk_4.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_5.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_6.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_7.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_8.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_9.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_10.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_11.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_12.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_13.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-Census/florida/data_chunk_14.csv
		Got reponse from Census API
Data chunk: /opt/data/GIS-C

FileNotFoundError: [Errno 2] No such file or directory: '/opt/data/GIS-Census/florida/data_chunk_21.csv'